In [ ]:
!pip install opencv-python-headless pillow torch torchvision pandas matplotlib

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import torch
import torchvision
from torchvision import transforms as T
from PIL import Image
import matplotlib.pyplot as plt
import cv2

# Faster R-CNN model
def get_model():

    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    model.eval()
    return model

# Function to transform input image
def get_transform():
    transform = T.Compose([
        T.ToTensor(),  # Convert PIL image or numpy array to a tensor
    ])
    return transform


# Function to perform inference on a single image
def detect_objects(image_path, model, transform):

    image = Image.open(image_path).convert("RGB")

    # Apply the transformation to the image
    image_tensor = transform(image).unsqueeze(0)  # Add batch dimension

    # Perform the inference
    with torch.no_grad():
        outputs = model(image_tensor)

    return outputs


def visualize_results(image_path, outputs, threshold=0.5):
    # Load the image using OpenCV
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Get the bounding boxes and labels
    boxes = outputs[0]['boxes'].cpu().numpy()
    scores = outputs[0]['scores'].cpu().numpy()

    # Draw the boxes on the image
    for i, box in enumerate(boxes):
        if scores[i] > threshold:
            # Convert box coordinates to integers
            box = box.astype(int)
            cv2.rectangle(image, (box[0], box[1]), (box[2], box[3]), color=(0, 255, 0), thickness=2)

    # Save the image with detections
    result_image_path = os.path.join('results', os.path.basename(image_path))
    cv2.imwrite(result_image_path, cv2.cvtColor(image, cv2.COLOR_RGB2BGR))


def process_dataset(dataset_dir, model, transform):
    # Create a results directory if it doesn't exist
    if not os.path.exists('results'):
        os.makedirs('results')

    # Iterate over all images in the directory
    for image_name in os.listdir(dataset_dir):
        image_path = os.path.join(dataset_dir, image_name)

        if image_path.endswith(('.png', '.jpg', '.jpeg')):
            print(f"Processing {image_name}...")
            outputs = detect_objects(image_path, model, transform)
            visualize_results(image_path, outputs)

def main():
    model = get_model()
    dataset_dir =  '/content/drive/MyDrive/Precog/Colab Notebooks/dataset'
    process_dataset(dataset_dir, model, get_transform())

if __name__ == "__main__":
    main()


Processing 87420.png...
Processing 47162.png...
Processing 02358.png...
Processing 85237.png...
Processing 76432.png...
Processing 97814.png...
Processing 47189.png...
Processing 81904.png...
Processing 86925.png...
Processing 78351.png...
Processing 90362.png...
Processing 87391.png...
Processing 96713.png...
Processing 78345.png...
Processing 81092.png...
Processing 46730.png...
Processing 78190.png...
Processing 80165.png...
Processing 92561.png...
Processing 02614.png...
Processing 84317.png...
Processing 86514.png...
Processing 81509.png...
Processing 82014.png...
Processing 98702.png...
Processing 98064.png...
Processing 93125.png...
Processing 98716.png...
Processing 76803.png...
Processing 94138.png...
Processing 82941.png...
Processing 85960.png...
Processing 91083.png...
Processing 85790.png...
Processing 90564.png...
Processing 01269.png...
Processing 89035.png...
Processing 83256.png...
Processing 02984.png...
Processing 87024.png...
Processing 01527.png...
Processing 93051

In [ ]:
import os
import torch
import torchvision
from torchvision import transforms as T
from PIL import Image
import matplotlib.pyplot as plt
import cv2
from collections import defaultdict


# COCO class labels
COCO_INSTANCE_CATEGORY_NAMES = [
    '__background__', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A', 'stop sign',
    'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant',
    'bear', 'zebra', 'giraffe', 'N/A', 'backpack', 'umbrella', 'N/A', 'N/A', 'handbag',
    'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat',
    'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'N/A', 'wine glass',
    'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli',
    'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'N/A',
    'dining table', 'N/A', 'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard',
    'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A', 'book', 'clock',
    'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
]


# Function to get the pretrained Faster R-CNN model
def get_model():
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    model.eval()
    return model

# Function to transform input image
def get_transform():
    transform = T.Compose([
        T.ToTensor(),
    ])
    return transform
# Function to perform inference on a single image
def detect_objects(image_path, model, transform):
    image = Image.open(image_path).convert("RGB")
    image_tensor = transform(image).unsqueeze(0)
    with torch.no_grad():
        outputs = model(image_tensor)
    return outputs

def visualize_and_catalog(image_path, outputs, object_catalog, threshold=0.5):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    boxes = outputs[0]['boxes'].cpu().numpy()
    scores = outputs[0]['scores'].cpu().numpy()
    labels = outputs[0]['labels'].cpu().numpy()

    for i, box in enumerate(boxes):
        if scores[i] > threshold:
            # Convert box coordinates to integers
            box = box.astype(int)
            cv2.rectangle(image, (box[0], box[1]), (box[2], box[3]), color=(0, 255, 0), thickness=2)
            label = labels[i]
            object_catalog[COCO_INSTANCE_CATEGORY_NAMES[label]] += 1

    result_image_path = os.path.join('results', os.path.basename(image_path))
    cv2.imwrite(result_image_path, cv2.cvtColor(image, cv2.COLOR_RGB2BGR))



def process_dataset(dataset_dir, model, transform):
    if not os.path.exists('results'):
        os.makedirs('results')

    object_catalog = defaultdict(int)

    for image_name in os.listdir(dataset_dir):
        image_path = os.path.join(dataset_dir, image_name)
        if image_path.endswith(('.png', '.jpg', '.jpeg')):
            print(f"Processing {image_name}...")
            outputs = detect_objects(image_path, model, transform)
            visualize_and_catalog(image_path, outputs, object_catalog)

    return object_catalog


def main():
    model = get_model()
    dataset_dir =  '/content/drive/MyDrive/Precog/Colab Notebooks/dataset'
    object_catalog = process_dataset(dataset_dir, model, get_transform())

    total_objects = sum(object_catalog.values())
    print(f"Total objects detected: {total_objects}")
    for obj, count in object_catalog.items():
        print(f"{obj}: {count} ({(count/total_objects)*100:.2f}%)")

if __name__ == "__main__":
    main()


Total objects detected: 0


In [ ]:
'''import cv2
import os

def load_images_from_folder(folder):
    images = []
    if not os.path.exists(folder):
        print(f"Folder '{folder}' does not exist.")
        return images

    for filename in os.listdir(folder):
        file_path = os.path.join(folder, filename)
        if os.path.isfile(file_path):
            img = cv2.imread(file_path)
            if img is not None:
                images.append(img)
            else:
                print(f"Failed to load image: {file_path}")
        else:
            print(f"Skipping non-file entry: {file_path}")

    if not images:
        print("No images were loaded.")
    return images

folder_path = '/content/drive/MyDrive/Precog/Colab Notebooks/dataset'
images = load_images_from_folder(folder_path)


In [ ]:
import torch
from torchvision import models, transforms

# Load a pre-trained object detection model (e.g., Faster R-CNN)
model = models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
model.eval()

# Define a transform to convert images to the format expected by the model
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
])

def detect_objects(image, model, transform):
    image_tensor = transform(image)
    image_tensor = image_tensor.unsqueeze(0)
    with torch.no_grad():
        outputs = model(image_tensor)
    return outputs[0]

detected_objects = [detect_objects(img, model, transform) for img in images]


In [ ]:
import pandas as pd

def extract_objects(detections, threshold=0.5):
    objects = []
    for detection in detections:
        for idx, score in enumerate(detection['scores']):
            if score > threshold:
                label = detection['labels'][idx].item()
                objects.append(label)
    return objects

object_catalog = [extract_objects(d) for d in detected_objects]
flattened_objects = [item for sublist in object_catalog for item in sublist]
object_df = pd.DataFrame(flattened_objects, columns=['object'])
object_distribution = object_df['object'].value_counts()


In [ ]:
# Assume you have a DataFrame with image paths and toxicity labels
# e.g., meme_data = pd.DataFrame({'image_path': ['path1', 'path2', ...], 'toxic': [1, 0, ...]})

def assign_toxicity_score(objects, toxic_objects, non_toxic_objects):
    score = 0
    for obj in objects:
        if obj in toxic_objects:
            score += 1
        elif obj in non_toxic_objects:
            score -= 1
    return score

# Get sets of objects from toxic and non-toxic memes
toxic_objects = set([obj for idx, obj_list in enumerate(object_catalog) if meme_data['toxic'][idx] == 1 for obj in obj_list])
non_toxic_objects = set([obj for idx, obj_list in enumerate(object_catalog) if meme_data['toxic'][idx] == 0 for obj in obj_list])

# Calculate toxicity scores for all memes
meme_data['toxicity_score'] = meme_data.apply(lambda row: assign_toxicity_score(object_catalog[row.name], toxic_objects, non_toxic_objects), axis=1)

# Simple classification: If score > 0, classify as toxic; else non-toxic
meme_data['predicted_toxic'] = meme_data['toxicity_score'] > 0


In [ ]:
from sklearn.metrics import accuracy_score, classification_report

accuracy = accuracy_score(meme_data['toxic'], meme_data['predicted_toxic'])
report = classification_report(meme_data['toxic'], meme_data['predicted_toxic'])

print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(report)
